In [2]:
#기틀 json의 유저 정보를 변경된 유저정보로 교체

# 부서번호 추가 코드

import pandas as pd
import json

# CSV 파일 읽기
df = pd.read_csv('인사정보_부서코드추가.csv', encoding='utf-8-sig')

# 사원명을 키로 하는 딕셔너리 생성 (부서코드 추가!)
employee_dict = {}
for _, row in df.iterrows():
    employee_dict[row['사원명']] = {
        'emailId': row['ID'],
        'deptName': row['부서'],
        'empNo': row['사원번호'] if pd.notna(row['사원번호']) else '',
        'positionName': row['직위'] if pd.notna(row['직위']) else '',
        'deptCode': row['부서코드'] if pd.notna(row['부서코드']) else ''  # 부서코드 추가!
    }

print(f"총 {len(employee_dict)}명의 사원 정보 로드 완료")

# JSON 파일 읽기
with open('any_docBodyImageSrcChange.json', 'r', encoding='utf-8') as f:
    approval_list = json.load(f)

print(f"총 {len(approval_list)}개의 결재 문서 로드 완료")

# 사원 정보 업데이트 함수 (empNo 제외) - 기존 함수
def update_employee_info(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            person_obj['emailId'] = employee_dict[name]['emailId']
            person_obj['deptName'] = employee_dict[name]['deptName']
            person_obj['positionName'] = employee_dict[name]['positionName']
    return person_obj

# deptCode 업데이트 함수 (새로 추가!)
def update_dept_code(person_obj):
    if person_obj and 'name' in person_obj:
        name = person_obj['name']
        if name in employee_dict:
            person_obj['deptCode'] = employee_dict[name]['deptCode']
    return person_obj

# 결재 목록 순회하며 정보 업데이트
updated_count = 0
dept_code_count = 0  # deptCode 업데이트 카운트

for doc in approval_list:
    # drafter 정보 업데이트 (emailId, deptName, positionName만) - 기존 로직
    if 'drafter' in doc:
        doc['drafter'] = update_employee_info(doc['drafter'])
        updated_count += 1
        # drafter deptCode 업데이트 (새로 추가!)
        doc['drafter'] = update_dept_code(doc['drafter'])
        dept_code_count += 1
    
    # approvals 리스트의 각 결재자 정보 업데이트 (emailId, deptName, positionName만) - 기존 로직
    if 'approvals' in doc:
        for approval in doc['approvals']:
            update_employee_info(approval)
            updated_count += 1
            # approvals deptCode 업데이트 (새로 추가!)
            update_dept_code(approval)
            dept_code_count += 1
    
    # referrers 리스트의 각 참조자 정보 업데이트 (empNo만) - 기존 로직
    if 'referrers' in doc:
        for referrer in doc['referrers']:
            if referrer and 'name' in referrer:
                name = referrer['name']
                if name in employee_dict:
                    referrer['empNo'] = employee_dict[name]['empNo']
                updated_count += 1
                # referrers deptCode 업데이트 (새로 추가!)
                update_dept_code(referrer)
                dept_code_count += 1
    
    # activities 리스트의 각 활동자 정보 deptCode 업데이트 (새로 추가!)
    if 'activities' in doc:
        for activity in doc['activities']:
            update_dept_code(activity)
            dept_code_count += 1

print(f"총 {updated_count}개의 사원 정보 업데이트 완료")
print(f"총 {dept_code_count}개의 deptCode 업데이트 완료")  # deptCode 업데이트 결과 출력

# 결과를 새로운 JSON 파일로 저장
with open('결재목록_업데이트완료.json', 'w', encoding='utf-8') as f:
    json.dump(approval_list, f, ensure_ascii=False, indent=2)

print("결재목록_업데이트완료.json 파일로 저장 완료")

# 업데이트 결과 샘플 확인
print("\n첫 번째 문서의 drafter 정보:")
print(json.dumps(approval_list[0]['drafter'], ensure_ascii=False, indent=2))

print("\n첫 번째 문서의 첫 번째 결재자 정보:")
if approval_list[0].get('approvals'):
    print(json.dumps(approval_list[0]['approvals'][0], ensure_ascii=False, indent=2))

print("\n첫 번째 문서의 첫 번째 activity 정보:")
if approval_list[0].get('activities'):
    print(json.dumps(approval_list[0]['activities'][0], ensure_ascii=False, indent=2))

총 156명의 사원 정보 로드 완료
총 3개의 결재 문서 로드 완료
총 4개의 사원 정보 업데이트 완료
총 16개의 deptCode 업데이트 완료
결재목록_업데이트완료.json 파일로 저장 완료

첫 번째 문서의 drafter 정보:
{
  "positionName": "상무",
  "deptName": "IP솔루션팀",
  "name": "편영수",
  "emailId": "yspyeon",
  "deptCode": "GB20"
}

첫 번째 문서의 첫 번째 결재자 정보:

첫 번째 문서의 첫 번째 activity 정보:
{
  "positionName": "",
  "deptName": "",
  "actionLogType": "DRAFT",
  "name": "편영수",
  "emailId": "",
  "type": "DRAFT",
  "actionDate": 1274939872000,
  "deptCode": "GB20",
  "actionComment": ""
}
